# Baseline model for batch monitoring example

In [5]:
import requests 
import pandas as pd 
import datetime 

from joblib import load, dump
from tqdm import tqdm 

from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [8]:
# download data from the internet 
files = [('green_tripdata_2022-02.parquet', '../data'), 
         ('green_tripdata_2022-01.parquet', '../data')]

print('Download files:')
for file, path in files:
    url=f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
    resp=requests.get(url, stream=True)
    save_path=f"{path}/{file}"
    with open(save_path, "wb") as handle:
        for data in tqdm(resp.iter_content(),
                         desc=f"{file}",
                         postfix=f"save to {save_path}",
                         total=int(resp.headers["Content-Length"])):
            handle.write(data)

Download files:


green_tripdata_2022-01.parquet: 100%|██████████| 1254291/1254291 [00:02<00:00, 474111.20it/s, save to ../data/green_tripdata_2022-01.parquet]


In [10]:
jan_data = pd.read_parquet('../data/green_tripdata_2022-01.parquet')

In [13]:
# create target
jan_data["duration_min"] = jan_data.lpep_dropoff_datetime - jan_data.lpep_pickup_datetime
jan_data.duration_min = jan_data.duration_min.apply(lambda x: float(x.total_seconds())/60)

In [14]:
# filter out outliers
jan_data = jan_data[(jan_data.duration_min >= 0) & (jan_data.duration_min <= 60)]
jan_data = jan_data[(jan_data.passenger_count > 0) & (jan_data.passenger_count <= 8)]

In [15]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [16]:
train_data = jan_data[:30000]
val_data = jan_data[30000:]

In [17]:
model = LinearRegression()

In [18]:
model.fit(train_data[num_features + cat_features], train_data[target])

LinearRegression()

In [19]:
train_preds = model.predict(train_data[num_features + cat_features])
train_data['prediction'] = train_preds

/var/folders/04/whwls5095tz8s5mxv15jpsvr0000gn/T/ipykernel_41630/3810346118.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['prediction'] = train_preds


In [22]:
val_preds = model.predict(val_data[num_features + cat_features])
val_data['prediction'] = val_preds

/var/folders/04/whwls5095tz8s5mxv15jpsvr0000gn/T/ipykernel_41630/3727490221.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data['prediction'] = val_preds


In [23]:
print(mean_absolute_error(train_data.duration_min, train_data.prediction))
      

3.82167686896383


In [25]:
# dump model and reference data 
with open('models/lin_reg.bin', 'wb') as f_out:
    dump(model, f_out)

FileNotFoundError: [Errno 2] No such file or directory: 'models/lin_reg.bin'